## Juntar, seleccionar atributos y generar promedios móviles de las Partículas PM

En este notebook se juntará los archivos de contaminación y meoteorología de cada año en un solo archivo, y también vamos a generar columnas de información temporal para cada mediciónn y por úlitmo se va a obtener el promedio móvil de el PM10 y PM2.5. 

- __Datos recibidos:__ Limpieza de datos inicial
- __Responsable:__ Daniel Bustillos
- __Contacto:__  juandaniel.bucam@gmail.com

__Pasos y descripción general del notebook__


Una vez que se tienen los datos limpios, es necesario aplicar una serie de manipulaciones:


1. Juntar los datos de las mediciones por año en un solo dataframe con el objetivo de agilizar su manipulación.


2. Se deshecharan las columnas "PBa","PMCO","UVA","UVB" y "WDR" debido a que éstos atributos no son distribuidos en tiempo real por lo que no es posible ocuparlas para el pronóstico. En el caso de "WDR" sí es compartido en tiempo real sin embargo, se concluyó que este no aporta a la descripción del fenómeno. 


3. Generar columnas con información temporal como la hora, día y mes en que se generó la medición, estos atributos son de ultilidad para utilizar los patrones temporaloes de temporalidad en el fenómeno de contaminación.


4. Se filtran los valores de PM10 y PM2.5 que salen del percentil .85. 


5. Por último generamos el atributo PM10mean y PM25mean que representarán el promedio móvil de las últimas 24 horas de los dos contaminantes.


In [1]:
import numpy as np
import pandas as pd
import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None 

## 1. 
### Juntar los datos de las mediciones por año en un solo dataframe con el objetivo de agilizar su manipulación.

Declaramos los nombres de todos los años con los que se generarán los modelos en una lista:

In [2]:
lista = ["cont_hora2016.csv", "cont_hora2017.csv", "cont_hora2018.csv"
         ,"cont_hora2015.csv","cont_hora2019.csv"]

Leemos los archivos y creamos un diccionario donde cada entrada es un el dataframe de un año en específico:

In [3]:
data_hora = {}
for elem in lista:
    data_hora[elem[9:13]] = pd.read_csv(str('/home/paw/DanielBustillos/contaminación/datasets/por_hora/'+ elem))
    data_hora[elem[9:13]] = data_hora[elem[9:13]].iloc[:,1:]

Creamos unn dataframe y juntamos todos los años en este:

In [4]:
df_append = pd.DataFrame(columns=data_hora["2019"].columns.tolist())
for key in data_hora:
    df_append = df_append.append(data_hora[key], ignore_index=True,sort=True)

Necesitamos convertir estos atributos a float:

In [5]:
df_append["PM2.5"] = df_append["PM2.5"].astype(float)
df_append["NO"] = df_append["NO"].astype(float)

Ordenemos el DF por fecha y estación de manera ascendente en ambos casos:

In [6]:
df_append = df_append.sort_values(['id_station','fecha'], ascending=[1, 1])

In [7]:
df_append.head(5)

,CO,NO,NO2,NOX,O3,PBa,PM10,PM2.5,PMCO,RH,...,TMP,UVA,UVB,WDR,WSP,dia,fecha,hora,id_station,mes
742865,0.6,14.0,21.0,35.0,4.0,NaN,84.0,NaN,NaN,82.0,...,9.0,NaN,NaN,358.0,1.8,1,2015-01-01,1,ACO,1
742892,0.8,29.0,21.0,50.0,5.0,NaN,110.0,NaN,NaN,86.0,...,8.4,NaN,NaN,344.0,1.9,1,2015-01-01,2,ACO,1
742920,0.8,37.0,21.0,58.0,6.0,NaN,140.0,NaN,NaN,91.0,...,7.8,NaN,NaN,354.0,1.7,1,2015-01-01,3,ACO,1
742948,1.0,39.0,18.0,58.0,7.0,NaN,131.0,NaN,NaN,93.0,...,7.1,NaN,NaN,25.0,1.4,1,2015-01-01,4,ACO,1
742975,1.0,38.0,16.0,54.0,6.0,NaN,151.0,NaN,NaN,95.0,...,6.2,NaN,NaN,334.0,1.1,1,2015-01-01,5,ACO,1


## 2. 
### También se deshecharan las columnas "PBa","PMCO","UVA","UVB" y "WDR" debido a que éstos atributos non se comparten en tiempo real por lo que no es posible ocuparlas para el pronóstico, en el caso de "WDR" si se comparte en tiempo real sin embargo, se concluyó que este no aporta a la descripción del fenómeno. 

Eliminamos los atributos mencionados:

In [8]:
df_append = df_append.drop(["PBa","PMCO","UVA","UVB","WDR"],axis=1)

## 3. 
### Generar columnas con información temporal como la hora, día y mes en que se generó la medición, estos atributos son de ultilidad para utilizar los patrones temporaloes de temporalidad en el fenómeno de contaminación.m

Arreglamos las fechas, vamos a generar columnas con el atributo hora,dia,mes.

In [9]:
df_append['dia'] = df_append['fecha'].astype(str).str[8:10]
df_append['mes'] =  df_append['fecha'].astype(str).str[5:7]
df_append['fecha'] = df_append['fecha'].astype(str).str[0:10]
df_append['dia'] = df_append['dia'].astype(str).astype(float)
df_append['mes'] = df_append['mes'].astype(str).astype(float)

Creamos una columna con toda esta información temporal, esta columna es de utilidad al ordenar y efectuar otras operaciones sobre el dataframe donde es necesario tener el orden de las mediciones bien definida.

In [10]:
df_append['fecha']  = pd.to_datetime([''.join([' '.join([df_append.loc[i, 'fecha'], 
                                                         str(df_append.loc[i, 'hora'])]),':00']) for i in df_append.index])
df_append['fecha'] = pd.to_datetime(df_append['fecha'], format='%Y-%m-%d %H:%M:%S')

In [11]:
df_append.head()

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes
742865,0.6,14.0,21.0,35.0,4.0,84.0,NaN,82.0,4.0,9.0,1.8,1.0,2015-01-01 01:00:00,1,ACO,1.0
742892,0.8,29.0,21.0,50.0,5.0,110.0,NaN,86.0,5.0,8.4,1.9,1.0,2015-01-01 02:00:00,2,ACO,1.0
742920,0.8,37.0,21.0,58.0,6.0,140.0,NaN,91.0,9.0,7.8,1.7,1.0,2015-01-01 03:00:00,3,ACO,1.0
742948,1.0,39.0,18.0,58.0,7.0,131.0,NaN,93.0,9.0,7.1,1.4,1.0,2015-01-01 04:00:00,4,ACO,1.0
742975,1.0,38.0,16.0,54.0,6.0,151.0,NaN,95.0,5.0,6.2,1.1,1.0,2015-01-01 05:00:00,5,ACO,1.0


Vamos a quedarnos con las celdas donde Pm10 y Pm2.5 son no nulas.

## 4. 
### Se filtran los valores de PM10 y PM2.5 que salen del percentil .85. 

Vemos que tenemos valores muy grandes, Jiang (2004) reportó que para valores mayores de 300 puntos, las mediciones no son confiables, vamos a quedarnos con mediciones menores a esta cota:

__Jiang, D., Zhang, Y., Hu, X., Zeng, Y., Tan, J., & Shao, D. (2004). Progress in developing an ANN model for air pollution index forecast. Atmospheric Environment, 38(40 SPEC.ISS.), 7055–7064. https://doi.org/10.1016/j.atmosenv.2003.10.066__

No queremos perder fechas, vamos a quedarnos con el valor anterior al outlier:

In [12]:
for i in range(len(df_append)):
    if df_append.loc[i,"PM10"] > 450:
        df_append.loc[i,"PM10"] = df_append.loc[i-1,"PM10"]
    if df_append.loc[i,"PM2.5"] > 450:
        df_append.loc[i,"PM2.5"] = df_append.loc[i-1,"PM2.5"]

## 5. 
### Por último generamos el atributo PM10mean y PM25mean que representarán el promedio móvil de las últimas 24 horas de los dos contaminantes.

Según la norma, la contingencia de PM10 y PM2.5 se considera cuando su promedio móvil de 24 horas excede 150 puntos, es necesario calcular el promedio móvil de cada contaminante para después poder pronosticarlo.

Volvemos a ordenar los datos de manera ascendente y ponemos la fecha cómo index:

In [13]:
df_append = df_append.sort_values(['fecha','id_station'], ascending=[1,1])

data_24 = df_append.set_index('fecha')

Ahora vamos a sacar el promedio de las ultimas 24 horas por estación para el PM10 y el PM25:

In [14]:
data_24 = data_24.groupby('id_station')[["PM10","PM2.5"]].rolling(24).mean().dropna().reset_index()

Renombramos estas columnas:m

In [15]:
data_24 = data_24.rename(columns={'PM10': 'PM10mean',"PM2.5":'PM25mean'})

##### Unimos el dataframe de los promedios móviles con el df original: 

In [16]:
data_day = pd.merge(df_append, data_24, on=["fecha","id_station"],how="left")

In [17]:
data_day.head(5)

,CO,NO,NO2,NOX,O3,PM10,PM2.5,RH,SO2,TMP,WSP,dia,fecha,hora,id_station,mes,PM10mean,PM25mean
0,0.6,23.0,22.0,45.0,3.0,60.0,NaN,73.0,2.0,9.1,1.4,1.0,2015-01-01,0,ACO,1.0,NaN,NaN
1,0.1,0.0,5.0,5.0,32.0,NaN,NaN,NaN,1.0,NaN,7.3,1.0,2015-01-01,0,AJM,1.0,NaN,NaN
2,NaN,NaN,NaN,NaN,15.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2015-01-01,0,AJU,1.0,NaN,NaN
3,1.0,32.0,28.0,60.0,3.0,28.0,NaN,NaN,2.0,NaN,NaN,1.0,2015-01-01,0,ATI,1.0,NaN,NaN
4,1.7,NaN,NaN,NaN,1.0,46.0,32.0,NaN,4.0,NaN,NaN,1.0,2015-01-01,0,CAM,1.0,NaN,NaN


### Guardamos

In [18]:
data_day.to_csv(str("./datos_entrenamiento.csv"),index=False)